## Arora-Ge Attack on LWE Demonstration

### 0. Dependencies and Directories

In [1]:
# Set root directory
import os

ROOT_DIR = "D:\Coding\CZ4010\Applied-Cryptography-Project"
os.chdir(ROOT_DIR)

In [2]:
import numpy as np
from math import comb
from pprint import pprint

from LWE_PKC import LWE_Encrypt, LWE_Decrypt
from utils import ascii_to_binary_list, binary_list_to_ascii

### 1. Testing out the Cryptosystem

We will encrypt a binary encoded message.

In [17]:
# Encode message with binary bits
message_ascii = "My username is RahulG1309."
message_binary = ascii_to_binary_list(message_ascii)

print(f"Message: {message_ascii}")
print(f"\nEncoded: {message_binary}")

Message: My username is RahulG1309.

Encoded: [0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0]


In [18]:
# LWE Parameters
n = 10
q = 9377
max_error = 1

# If we have sufficient samples we can recover the secret.
m = 500

# Initialize PKC with parameters
lwe_d = LWE_Decrypt(n=n, q=q, max_error=max_error, list_size=m)

In [19]:
# Public Keys
A_list, T_list, q, max_error = lwe_d.get_public_keys()

print("A_list:")
pprint(A_list)

print("\nT_list:")
pprint(T_list)

A_list:
array([[1132, 7594, 6908, ..., 6741, 3223, 5375],
       [1192, 7370, 7052, ..., 4012, 8053, 2826],
       [1360, 6901,  664, ..., 8362, 5230, 1106],
       ...,
       [3137, 5466,  147, ..., 2225, 5611, 8863],
       [9257, 4537, 8070, ..., 1109, 7374, 9354],
       [ 570, 2046, 2581, ..., 2952, 3221, 7301]])

T_list:
array([6252, 2619, 7819, 7247, 1926, 5484, 1486, 2090, 9229,  476, 4957,
       5203, 2843, 9222, 8913, 6704, 8756, 7358, 6188, 4062, 7471,  613,
       8726,  940, 6244, 7730, 1796, 9131, 7239, 4886, 3158, 8487, 7052,
       8847, 1210, 4833,  520, 1401, 2795, 7618, 2129, 6586,  317, 3692,
        756, 4018, 4851, 6681, 1393,  664, 4676, 5325, 7839, 8164,  856,
       1284, 7517,  265, 6292, 8515,  564, 2981, 9216, 5186, 6690, 9278,
       4469, 2884, 5585, 7141, 6467, 6460, 3575, 7717, 3919, 7198,  951,
       1302, 4968, 5586, 5190, 8512, 9324, 5557,  655, 6617, 4179, 8478,
       5065, 8288, 5784, 5270, 7068, 9149, 6962, 5442,  867, 7770, 6724,
       3252, 

In [20]:
# Encrypt Message
lwe_e = LWE_Encrypt(A_list, T_list, q, max_error)
A_new, T_send = lwe_e.encrypt_message(message_binary)

print("A_new:")
pprint(np.array(A_new))

print("\nT_send:")
pprint(np.array(T_send))

A_new:
array([[1466, 4072, 6405, ..., 5393, 2052, 5893],
       [ 913, 2381, 8700, ..., 7182, 5493, 6532],
       [4148, 6122, 6127, ..., 8428, 7996, 5311],
       ...,
       [8121, 2244, 3143, ..., 6652, 5777, 8197],
       [3377, 4934, 3555, ..., 8359, 1010, 2402],
       [5967, 8200, 7778, ..., 7368, 4679, 1872]], dtype=int32)

T_send:
array([3697, 3395, 2021, 6043, 4167, 1611, 3694, 1272, 5485, 4499, 2999,
       9317, 5389,  676, 2380,  874, 3487, 9053, 8811, 7241, 1084, 7331,
       7695, 2226, 7424, 2694, 5420, 5139, 3562, 2466, 6897, 4828, 3880,
       2538, 3329, 3567,  678, 3567,  534, 5546, 1005, 7709,  833, 4730,
       5345, 7383, 5775, 7683, 9300,  244, 2075, 5410, 2840, 5295, 6828,
       3222, 5075, 1655,   48, 7828,  372, 2852,  611, 8682, 7801, 7693,
       4032, 7772,  524,  420, 3512, 1992, 1911,  152, 1102, 3206, 3561,
       6105, 4852,  243, 3912,  255, 5597, 2684, 1715, 8816, 7812, 7138,
       6650, 3184, 8776, 5664, 7551, 3098, 3968, 5461, 8507, 3914, 8074,
 

In [21]:
# Decrypt Message
decrypted_messge_binary = lwe_d.decrypt_message(A_new, T_send)
decrypted_messge_ascii = binary_list_to_ascii(decrypted_messge_binary)
print(f"Decrypted Message: {decrypted_messge_binary}")
print(f"\nDecoded: {decrypted_messge_ascii}")

Decrypted Message: [0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0]

Decoded: My username is RahulG1309.


### 2. Arora-Ge Algebraic Attack

As part of LWE, the errors are drawn from a finite set, a fact which makes
it possible to attack LWE and recover the secret without knowing the secret
key. Arora and Ge (Princeton) proposed an attack in 2011 that leverages this property using
algebraic methods to recover the secret.

<b>References,</b>
<br> `https://users.cs.duke.edu/~rongge/LPSN.pdf`
<br> `https://eprint.iacr.org/2014/1018.pdf`
<br> `https://people.csail.mit.edu/vinodv/CS294/lecture2.pdf`

If the error distribution is truncated and takes values in the set $\{-T, ... , -1, 0, 1, ... , T\} \implies |E| = 2T + 1$ where, $T = \text{max\_error}$

We can recover the secret `s` from `A_list` and `b_list` if we have enough LWE samples <A, b>.

Precisely we require $m >= {n + |E| \choose |E|}$ LWE samples to solve the system of polynomials by linearization.

In [1]:
# Set root directory
import os

ROOT_DIR = "D:\Coding\CZ4010\Applied-Cryptography-Project"
os.chdir(ROOT_DIR)

In [2]:
import numpy as np
from math import comb
from pprint import pprint

from LWE_PKC import LWE_Encrypt, LWE_Decrypt
from utils import ascii_to_binary_list, binary_list_to_ascii

from itertools import product
from collections import defaultdict
from sympy import symbols, Matrix, GF, init_printing
from sympy.polys.matrices import DomainMatrix

In [3]:
# LWE Parameters
n = 10
q = 9377
max_error = 1

m = 300

# If we have sufficient samples we can recover the secret.
E = 2*max_error + 1
m = comb(n+E, E) - 1

# Initialize PKC with parameters
lwe_d = LWE_Decrypt(n=n, q=q, max_error=max_error, list_size=m)

In [4]:
# The secret is randomly initalized in the class
A_list, b_list, q, max_error = lwe_d.get_public_keys()
print(f"Randomly Initalized Secret: {lwe_d.secret}")

Randomly Initalized Secret: [490, 8393, 689, 1211, 1741, 8683, 7163, 808, 3643, 3313]


In [5]:
print(f"Cardinality of error set: {E}")
print(f"Number of LWE samples: {len(A_list)} >= {comb(n+E, E)-1}")

Cardinality of error set: 3
Number of LWE samples: 285 >= 285


In [6]:
# Define the secret vector
secret_vector = symbols(f'x1:{n+1}')
secret_vector

(x1, x2, x3, x4, x5, x6, x7, x8, x9, x10)

In [7]:
# Construct the polynomials for each LWE instance <A, b>
polynomials_over_Zq = []
error_set = [i for i in range(-max_error, max_error+1)] 

for A, b in zip(A_list, b_list):
    # Initalize the polynomial term to the identity polynomial of the finite field
    polynomial_over_Zq = GF(q)[secret_vector](1)

    for e in error_set:
        # Multiply each variable by its corresponding weight
        weighted_polynomial = sum(w * var for w, var in zip(A, secret_vector))

        # Construct the weighted polynomial (this is the AT*s term in the equation)
        weighted_secret_polynomial_over_Zq = GF(q)[secret_vector](weighted_polynomial)

        # Complete the term (b - AT*s - e)
        term = b - weighted_secret_polynomial_over_Zq - e

        # Accumulate the product
        polynomial_over_Zq = polynomial_over_Zq * term
    
    polynomials_over_Zq.append(polynomial_over_Zq)

# Sanity check
assert len(polynomials_over_Zq) == len(A_list)

Now that we have the polynomials, let us solve them by linearization.

In [8]:
def generate_tuples(n, d):
    """
    Helper function that determines all tuples that represent the monomials in the polynomials constructed from an LWE sample.
    """
    # Use itertools.product to generate all tuples
    tuples = list(product(range(d + 1), repeat=n))
    
    # Remove tuples that have a degree > d
    tuples_pruned = [x for x in tuples if sum(x) <= d]

    return tuples_pruned

In [9]:
coefficients_dicts = []
tuples = generate_tuples(n, E)

for polynomial in polynomials_over_Zq:
    coefficients_dict = defaultdict(int, {key: 0 for key in tuples})

    for term_key, coeff in polynomial.terms():
        coefficients_dict[term_key] = int(coeff) # NOTE: Converting to int!!!

    coefficients_dicts.append(coefficients_dict)

In [10]:
row_order = []

# Let's keep the degree 1 terms up front for convenince
for i in range(n):
    term = [0]*n
    term[i] = 1
    row_order.append(tuple(term))

# We want the secret at the start and the constant term at the end
seen = set(row_order)
constant_term_key = tuple([0]*n)
candidates = generate_tuples(n, E)

for x in candidates:
    if x not in seen and x != constant_term_key:
        row_order.append(x)

# Ensure the constant term is at the end
row_order.append(constant_term_key)

# Sanity check
assert len(row_order) == len(candidates)

In [11]:
# Construct the coefficient matrix
coefficient_matrix = []
rhs = []

for coeff_dict in coefficients_dicts:
    row = []

    for key in row_order[:-1]:
        row.append(coeff_dict[key])
    
    # Append row
    coefficient_matrix.append(row)

    # Append the rhs
    negative_constant_term = -coeff_dict[row_order[-1]]
    rhs.append(negative_constant_term)

Let us take a look at the matrices before solving them over the Zq domain.

In [12]:
# Solving linear system using DomainMatrix
m = Matrix(coefficient_matrix)
b = Matrix(rhs)

# Convert matrices to finite field of order q (q is prime):
K = GF(q, symmetric=False)
dm = DomainMatrix.from_Matrix(m).convert_to(K)
bm = DomainMatrix.from_Matrix(b).convert_to(K)

# Print shape of system of equations
print(dm.shape)
print(bm.shape)

# Solve and convert back to an ordinary Matrix:
solution_vector = dm.lu_solve(bm).to_Matrix()

(285, 285)
(285, 1)


In [13]:
print(f"Randomly Initalized Secret:\t\t{lwe_d.secret}")
print(f"Secret obtained from Arora-Ge Attack:\t{solution_vector[:n]}")

Randomly Initalized Secret:		[490, 8393, 689, 1211, 1741, 8683, 7163, 808, 3643, 3313]
Secret obtained from Arora-Ge Attack:	[490, 8393, 689, 1211, 1741, 8683, 7163, 808, 3643, 3313]


In [14]:
print(f"Did we correctly determine the secret? \n{lwe_d.secret == solution_vector[:n]}")

Did we correctly determine the secret? 
True


Works pretty well for `n <= 12` and `max_error = 1 or 2`. Basically ensure `m <= 1000` else it takes VERY long to run and will most likely go out of memory too.


### Thank you!
